In [10]:
import sys
sys.path.append('../')
#import functions
#exec(open('./functions.py').read())
%run functions.py
import demoji
#demoji.download_codes()

[nltk_data] Downloading package stopwords to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
def classify_bots(df):
    #collect all the account names related to bots and scripts
    names = []
    pat = r'(bot\s|script|bot_)'
    for name in df.name.unique():
        match = re.findall(pat, name, re.IGNORECASE) 
        if len(match) > 0:
            names.append(name)
    #look for a key words to identify a tweet related to bots and scripts
    pattern = r"(script|bot\s|bots\s|bot_|cook|cop^e|destroy|proxy)"
    df['bot'] = False
    for i, row in df.iterrows():
        match = re.findall(pattern, row.tweet, re.IGNORECASE) 
        if len(match) > 0 or row.isin(names)['name'] == True:
            df['bot'].loc[i] = True
    return df

In [12]:
#load labelled data (generated in file 2.preprocessing_en)
data = pickle.load(open('data_n_models/de_df_labelled.pkl', "rb"))
data.head()
#filter out bot related data
data = classify_bots(data)
data = data[data['bot']==False]
data = data.drop_duplicates(subset=['lemma'])
data.head()

C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,clean,clean_vader,compound,lemma,bot
0,1,1079317443523297280,12303,Boulder667,"@spdde @LadyBitchRay1 Viel Spaß, damit eine Wa...",0,"Liverpool, England",07-Mar-2021,116,False,1.0,1.368148e+18,zalando,de,viel spaß damit eine wahl gewinnen stellt euch...,"Viel Spaß, damit eine Wahl gewinnen. Stellt Eu...",-0.5994,viel spaß damit einen wahl gewinnen stellen si...,False
1,4,310342425,11476,AndyBaldauf,@dicecco @Zalando aber erst wenn sie wieder dü...,0,Thurgau | St. Gallen | Zürich,06-Mar-2021,1848,False,0.0,1.368146e+18,zalando,de,aber erst wenn sie wieder dürfen verspielt,aber erst wenn sie wieder dürfen :-P,0.4995,aber erst wenn ich wieder dürfen verspielen,False
2,10,793418126532734976,521,MILESmobility,Good News! Alle interessierten MitarbeiterInne...,0,"Berlin, Deutschland",05-Mar-2021,766,False,8.0,NaN,zalando,de,good news alle interessierten mitarbeiterinnen...,Good News! Alle interessierten MitarbeiterInne...,-0.7568,good news all interessieren mitarbeiterinnen d...,False
3,11,1119231287078858752,23,pici1303,@Zalando Ich weiß dass der Schein da rein muss...,0,,04-Mar-2021,0,False,0.0,1.367531e+18,zalando,de,ich weiß dass der schein rein muss aber ist mi...,Ich weiß dass der Schein rein muss aber was is...,0.0000,ich weiß dass der schein rein muss aber sein m...,False
4,12,1119231287078858752,23,pici1303,@Zalando Ich hab mich nur gefragt wofür dieser...,0,,04-Mar-2021,0,False,0.0,1.367528e+18,zalando,de,ich hab mich nur gefragt wofür dieser extra co...,Ich hab mich nur gefragt wofür dieser extra Co...,0.0000,ich hab sich nur fragen wofür dies extra code ...,False


In [13]:
# change compound from continous to categorical variable
data["compound"] = np.where(data["compound"] > 0, 1, data["compound"])
data["compound"] = np.where(data["compound"] < 0, -1, data["compound"])

In [14]:
shuffled_df = data.sample(frac=1, random_state= 42)

In [45]:
shuffled_df.iloc[0:500]

,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,clean,clean_vader,compound,lemma,bot
2274,104,1364996553581219842,1393,yasminnn2903,@bxrxak hä zum Beispiel wie ASOS oder so kenns...,0,aus deiner mutter,07-Apr-2021,227,False,1.0,1.379782e+18,asos,de,zum beispiel wie asos oder kennst nicht oder wie,zum Beispiel wie ASOS oder kennst nicht oder wie,0.0,zum beispiel wie asos oder kennen nicht oder wie,False
57,79,1358102479657967618,3586,localbunnybitch,@whosmartinn Achsooooo ja hm\nAlso bei vinted ...,0,she/her,01-Mar-2021,207,False,2.0,1.366471e+18,asos,de,achsooooo hmalso bei vinted sind immer ganz ni...,Achsooooo hmAlso bei vinted sind immer ganz ni...,1.0,achsooooo hmalso bei vinted sein immer ganz ni...,False
2673,256,1027214204,67514,JudicatorPredo,"@undalles0yeah @isi_peazy Weiß gar nicht, ob Z...",0,Mitte des Flusses,12-Apr-2021,314,False,1.0,1.381686e+18,zalando,de,weiß gar nicht zalando überhaupt schwarze zahl...,"Weiß gar nicht, Zalando überhaupt schwarze Zah...",0.0,weiß gar nicht zalando überhaupt schwarze zahl...,False
1400,21,1233373817407496192,26199,ButterflyAbuse,@the_fesxh @OlliL @Zalando Geht nich,0,"Rostock, Deutschland",24-Mar-2021,10895,False,3.0,1.374828e+18,zalando,de,geht nich,Geht nich,0.0,gehen nich,False
1737,16,1360904902202056707,54,Selinasahar,Was sich wohl die Leute bei der Bank denken di...,0,,31-Mar-2021,8,False,0.0,NaN,zalando,de,sich wohl die leute bei der bank denken die je...,Was sich wohl die Leute bei der Bank denken di...,-1.0,sich wohl der leute bei der bank denken der je...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260,61,1289565265,610555,Ratings_via_FL,▶ dpa-AFX: Berenberg hebt Ziel für Asos auf 70...,0,37574,09-Apr-2021,488,False,0.0,NaN,asos,de,dpa afx berenberg hebt ziel für asos auf pence...,dpa-AFX: Berenberg hebt Ziel für Asos auf Penc...,-1.0,dpa afx berenberg heben ziel für asos auf penc...,False
419,21,44144586,172342,godmodetrader,"#Zalando - Das war knapp, das war richtig knap...",0,"München, Deutschland",10-Mar-2021,9278,False,2.0,NaN,zalando,de,zalando das war knapp das war richtig knapp bö...,"Zalando Das war knapp, das war richtig knapp b...",-1.0,zalando der sein knapp der sein richtig knapp ...,False
702,38,1289565265,605853,Ratings_via_FL,▶ RBC belässt Zalando auf 'Outperform' - Ziel ...,0,37574,16-Mar-2021,493,False,0.0,NaN,zalando,de,rbc belässt zalando auf outperform ziel euro n...,RBC belässt Zalando auf 'Outperform' Ziel Euro...,-1.0,rbc belässt zalando auf outperform ziel euro n...,False
1203,12,713431689767297026,244,timbabst,@levisimp666 Leider hast du nicht bei Zalando ...,0,,24-Mar-2021,41,False,0.0,1.374488e+18,zalando,de,leider hast nicht bei zalando mitgemacht hätte...,Leider hast nicht bei Zalando mitgemacht ..da ...,0.0,leider haben nicht bei zalando mitmachen haben...,False


In [87]:
df = pd.read_csv('data_n_models/de_man_labelled_data_new.csv', sep = ';').drop('index', axis= 1)

In [88]:
df.head()

,Unnamed: 0,Sentiment
0,2087,-1
1,287,0
2,2127,0
3,94,0
4,250,1


In [89]:
shuffled_df[shuffled_df.index == 2087]

,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,clean,clean_vader,compound,lemma,bot
2087,1,1200528183164067841,25959,ToskaBoy,@FriedhofchiIIer Boohoo soll doch in Rente gehen,0,,08-Apr-2021,251,False,3.0,1.380103e+18,boohoo,de,boohoo soll doch rente gehen,Boohoo soll doch Rente gehen,0.0,boohoo soll doch rente gehen,False


In [101]:
merged_df = pd.merge(shuffled_df, df, right_on='Unnamed: 0', left_index=True, how = 'inner')

In [102]:
merged_df = merged_df.drop(['Unnamed: 0'], axis = 1)

In [99]:
merged_df = merged_df.drop('index', axis = 1).reset_index().drop('index', axis = 1).reset_index()

In [104]:
merged_df.Sentiment = merged_df.Sentiment.astype('float64')

In [105]:
merged_df

,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,clean,clean_vader,compound,lemma,bot,Sentiment
39,79,1358102479657967618,3586,localbunnybitch,@whosmartinn Achsooooo ja hm\nAlso bei vinted ...,0,she/her,01-Mar-2021,207,False,2.0,1.366471e+18,asos,de,achsooooo hmalso bei vinted sind immer ganz ni...,Achsooooo hmAlso bei vinted sind immer ganz ni...,1.0,achsooooo hmalso bei vinted sein immer ganz ni...,False,1.0
107,256,1027214204,67514,JudicatorPredo,"@undalles0yeah @isi_peazy Weiß gar nicht, ob Z...",0,Mitte des Flusses,12-Apr-2021,314,False,1.0,1.381686e+18,zalando,de,weiß gar nicht zalando überhaupt schwarze zahl...,"Weiß gar nicht, Zalando überhaupt schwarze Zah...",0.0,weiß gar nicht zalando überhaupt schwarze zahl...,False,0.0
144,155,33618241,7969,cyclist_city,@Zalando_Press @Vizions2017 Super! Warum nicht...,0,"Berlin, Germany",02-Mar-2021,1223,False,3.0,NaN,zalando,de,super warum nicht eurer heimat berlin,Super! Warum nicht eurer Heimat Berlin?,1.0,super warum nicht eurer heimat berlin,False,1.0
52,86,23059415,14583,birgitzz,"Von wegen, Umsatzeinbruch während Corona --&gt...",2,"Zurich, Switzerland",15-Mar-2021,1280,False,2.0,NaN,zalando,de,von wegen umsatzeinbruch während corona zaland...,"Von wegen, Umsatzeinbruch während Corona --&gt...",0.0,von wegen umsatzeinbruch während corona zaland...,False,1.0
196,25,1250097593612472322,1868,zItzMatze,"@aycaxkaya asos, die hatten da zwei übelst nic...",0,bei mir zu hause,04-Apr-2021,31,False,1.0,1.378645e+18,asos,de,asos die hatten zwei übelst nice tshirts,"asos, die hatten zwei übelst nice tshirts",-1.0,asos der haben zwei übelst nice tshirts,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,104,943469642101780480,494,DennisFromm2,"Statt 360-Grad-Feedback, in der Praxis 360-Gra...",0,,09-Apr-2021,235,False,0.0,NaN,zalando,de,statt grad feedback der praxis grad überwachun...,"Statt -Grad-Feedback, der Praxis -Grad-Überwac...",0.0,statt grad feedback der praxis grad überwachun...,False,-1.0
84,136,772507713993662464,4075,LXSixty,@WJuleez @Zalando Ich habe etwas vor einer Woc...,0,Österreich,02-Apr-2021,79,False,0.0,1.377922e+18,zalando,de,ich habe etwas vor einer woche bestellt und mi...,Ich habe etwas vor einer Woche bestellt und mi...,-1.0,ich habe etwas vor einer woche bestellen und s...,False,-1.0
28,37,1369735338810294272,1047,Analplug46,Was gibt's neues bei Asos,0,,16-Apr-2021,79,False,1.0,NaN,asos,de,gibt neues bei asos,Was gibt's neues bei Asos,0.0,geben neu bei asos,False,0.0
41,29,1196470239325765632,829,maryjanesenior,TW Body\nich: struggle gerade mega mit meinem ...,0,,24-Mar-2021,52,False,3.0,NaN,zalando,de,bodyich struggle gerade mega mit meinem körper...,Bodyich: struggle gerade mega mit meinem körpe...,-1.0,bodyich struggle gerade mega mit mein körper e...,False,-1.0


In [106]:
# column 'compound' from vader -> predicted values
# column 'Sentiment' own labelled data -> true values
# in order to determine the correctness of vader's sentiment evaluation we can use the confusion matrix to determine the 

pd.crosstab(merged_df.compound, merged_df.Sentiment, rownames=['compound'], colnames=['Sentiment'])


Sentiment,-1.0,0.0,1.0
compound,,,
-1.0,38,23,17
0.0,21,61,22
1.0,4,5,14


True Positive (TP) 

    The predicted value matches the actual value
    The actual value was positive and the model predicted a positive value

True Negative (TN) 

    The predicted value matches the actual value
    The actual value was negative and the model predicted a negative value

False Positive (FP) – Type 1 error

    The predicted value was falsely predicted
    The actual value was negative but the model predicted a positive value
    Also known as the Type 1 error

False Negative (FN) – Type 2 error

    The predicted value was falsely predicted
    The actual value was positive but the model predicted a negative value
    Also known as the Type 2 error
